In [ ]:
# 20170415 attempt: case.518.com -> 500 Internal Server Error

In [179]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import config

account = config.account
pwd = config.pd

In [228]:
def get_518_login_session(br):
    # fill info to get login session
    br.get('http://case.518.com.tw/member-logout.html')
    br.get('https://case.518.com.tw/member-login.html')
    br.find_element_by_id('account').send_keys(account)
    br.find_element_by_id('pwd').send_keys(pwd)
    br.find_element_by_xpath('//*[@id="content-inner"]/form/div/p[4]/input').click()

    br.get('http://case.518.com.tw/casepage-index.html')
    
    s = requests.Session()
    for c in br.get_cookies():
        s.cookies.set(c['name'], c['value'])
    return s

# br = webdriver.Chrome('./drivers/chromedriver_mac64')
br = webdriver.PhantomJS('./drivers/phantomjs-2.1.1-macosx/bin/phantomjs')
s = get_518_login_session(br)
br.quit()

In [229]:
def get_case_list_df_by_page_url(page_url):
    res = s.get(page_url)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    # parse needed data
    titles = [a.text.strip() for a in soup.select('li.title div a')]
    links = [a.get('href') for a in soup.select('li.title div a')]
    propose_amount = [int(a.text.strip()) for a in soup.select('li.active a')]
    budgets = [s.text.strip() for s in soup.select('li.budget span')]
    views = [int(s.text.strip()) for s in soup.select('li.view span')]
    locations = [s.text.strip() for s in soup.select('li.status span')]
    post_dates = [s.text.strip() for s in soup.select('time')]

    df = pd.DataFrame({
        'title': titles,
        'link': links,
        'propose_amount': propose_amount,
        'budget': budgets,
        'view': views,
        'location': locations,
        'post_date': post_dates
    })
    time.sleep(1)
    return df

In [230]:
def get_final_page(url):
    soup = BeautifulSoup(s.get(url).text, 'html.parser')
    return int(soup.select_one('span.pagecountnum span').text\
                                                        .split('/')[1]\
                                                        .strip())

web_section_url = 'http://case.518.com.tw/casepage-index.html?srh=KWCC20MMTA7MM'
final_page = get_final_page(web_section_url)

frame = []
pages = range(1, final_page+1)
for page in tqdm(pages):
    page_url = web_section_url.replace('.html?', 
                                       '.html?page={}&'.format(page))
    _df = get_case_list_df_by_page_url(page_url)
    frame.append(_df)
    
basic_info_df = pd.concat(frame).reset_index(drop=True)

100%|██████████| 32/32 [00:49<00:00,  1.46s/it]


In [231]:
def get_case_info_by_link(link):
    """
    return tuple contains
    link, content, location, office, time_limit and owner_positive
    """
    res = s.get(link)
    soup = BeautifulSoup(res.text, 'html.parser')

    content = soup.select_one('dd.conttext p').text.strip()
    try:
        office = soup.select('dd.loc')[1].text.strip()
    except Exception as e:
        office = np.nan
        
    time_limit = soup.select_one('dd.date').text.strip()
    owner_positive = soup.select_one('p.positive').text.strip()

    time.sleep(0.5)
    return (link, content, office, time_limit, owner_positive)

rows = []
for link in tqdm(basic_info_df['link']):
    row = get_case_info_by_link(link)
    rows.append(row)

advance_info_columns = ['link', 'content', 'office', 
                        'time_limit', 'owner_positive']
advance_info_df = pd.DataFrame(rows, columns=advance_info_columns)
rs_df = basic_info_df.merge(advance_info_df, on='link')


100%|██████████| 621/621 [17:05<00:00,  1.83s/it]


In [227]:
rs_df.to_csv('../data/case518_web.tsv', sep='\t',
                                          index=False, 
                                          encoding='utf-8')

rs_df.to_json('../data/case518_web.json', force_ascii=False, 
                                            orient='records')

In [260]:
rs_df

,budget,link,location,post_date,propose_amount,title,view,content,office,time_limit,owner_positive
0,1萬~5萬,http://case.518.com.tw/casepage-detail-124218....,不拘,2017-04-22,1,軟體購物入口網站設計與切板案,132,"1.案件說明：線上購物入口網站設計與切板案,如題工作內容有\r\n 1.規劃每個版面配置與...",在家工作 / 在自設公司、工作室接案 / 須定期至指定公司,60 天 【一般案件】,中
1,10萬~30萬,http://case.518.com.tw/casepage-detail-124215....,台中市,2017-04-22,6,3D創網站版面及前台後台設計,164,"1.案件說明：公司決定建立一個銷售網站,需要懂網頁設計的人才,\r\n已經有功能上的方向及想...",須定期至指定公司,30 天 【一般案件】,中
2,1萬~5萬,http://case.518.com.tw/casepage-detail-124205....,不拘,2017-04-22,6,為Wordpress網頁設計及排版,184,配合時間：預計2017年06月底前完成驗收\r\n配合地點：在家作業，以E-mail/Lin...,在家工作,30 天 【長期合作】 (註:尋找日後配合人選),中
3,5萬~10萬,http://case.518.com.tw/casepage-detail-123996....,不拘,2017-04-22,4,彩卷網站與APP開發,530,1.案件說明：彩卷網站程式與APP(IOS&Android)開發，需一體化搭配網站切版設計風...,在家工作 / 在自設公司、工作室接案,30 天 【長期合作】 (註:尋找日後配合人選),中
4,1萬~5萬,http://case.518.com.tw/casepage-detail-123609....,不拘,2017-04-22,12,寵物用品 FB粉絲團經營 增加真人粉絲,1238,1.案件說明：\r\n寵物用品 FB粉絲團經營 增加真人粉絲\r\n商品化題操作企劃: 結合...,在家工作 / 在自設公司、工作室接案,14 天 【長期合作】 (註:尋找日後配合人選),中
5,5萬~10萬,http://case.518.com.tw/casepage-detail-124198....,台北市,2017-04-21,1,網站RWD及App UI/UX,246,1.案件說明：配合專案切版、設計App UI/UX\r\n2.配合時間：長期配合\r\n3....,在家工作 / 在自設公司、工作室接案 / 須定期至指定公司,60 天 【一般案件】,低
6,5萬~10萬,http://case.518.com.tw/casepage-detail-124177....,台北市,2017-04-21,2,徵求網站RWD化 及 APP UX/UI,291,1.案件說明：徵求網站RWD化 及 APP UX/UI設計\r\n2.配合時間：面議 (日後...,在自設公司、工作室接案,30 天 【長期合作】 (註:尋找日後配合人選),高
7,1萬~5萬,http://case.518.com.tw/casepage-detail-124176....,台北市,2017-04-21,2,日本電商商品上架及廣告投放整理,419,案件：日本電商商品上架及廣告投放整理\r\n金額：1萬~5萬\r\n徵求期限：30天《長期合...,在家工作 / 在自設公司、工作室接案,30 天 【長期合作】 (註:尋找日後配合人選),高
8,1萬~5萬,http://case.518.com.tw/casepage-detail-124163....,不拘,2017-04-21,4,歐克靈次氯酸抗菌液製造機產品網頁,291,"1.案件說明：歐克靈次氯酸抗菌液製造機產品網頁\r\n參考我們部落格內容,將內容做成網站(不...",在家工作 / 在自設公司、工作室接案,30 天 【急件】,中
9,5萬~10萬,http://case.518.com.tw/casepage-detail-124162....,台北市,2017-04-21,7,PHP 程式設計師,283,1.案件說明：產品、專案、系統開發\r\n2.配合時間：長期合作\r\n3.配合地點：台北市...,在家工作 / 須定期至指定公司,60 天 【長期合作】 (註:尋找日後配合人選),低
